In [1]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

In [46]:
data = pd.read_csv('../../lab-import-export/vehicles/vehicles_messy.csv')

In [47]:
#Examining Data for Potential Issues:
data.head()

,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,...,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb
0,15.695714,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
1,29.964545,0.0,0.0,0.0,9,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
2,12.207778,0.0,0.0,0.0,23,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
3,29.964545,0.0,0.0,0.0,10,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
4,17.347895,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0


In [7]:
#Missing Values:

null_cols = data.isnull().sum()
null_cols[null_cols > 0]

cylinders       123
displ           120
drive          1189
eng_dscr      15403
trany            11
guzzler       35562
trans_dscr    22796
tCharger      32657
sCharger      37177
atvType       34771
fuelType2     36435
rangeA        36440
evMotor       37281
mfrCode       30818
c240Dscr      37806
c240bDscr     37807
startStop     31705
dtype: int64

In [8]:
#Removing columns with more than 10000 fields with missing or null values:

drop_cols = list(null_cols[null_cols > 10000].index)
data = data.drop(drop_cols, axis=1)

In [9]:
#Examine columns with similar missing value count and try finidng correlation:

null_displ = data[(data['displ'].isnull()==True)]
null_displ = null_displ[['year', 'make', 'model', 'trany', 'drive','fuelType','cylinders', 'displ']]
null_displ

,year,make,model,trany,drive,fuelType,cylinders,displ
7138,2000,Nissan,Altra EV,NaN,NaN,Electricity,NaN,NaN
7139,2000,Toyota,RAV4 EV,NaN,2-Wheel Drive,Electricity,NaN,NaN
8143,2001,Toyota,RAV4 EV,NaN,2-Wheel Drive,Electricity,NaN,NaN
8144,2001,Ford,Th!nk,NaN,NaN,Electricity,NaN,NaN
8146,2001,Ford,Explorer USPS Electric,NaN,2-Wheel Drive,Electricity,NaN,NaN
8147,2001,Nissan,Hyper-Mini,NaN,NaN,Electricity,NaN,NaN
9212,2002,Toyota,RAV4 EV,NaN,2-Wheel Drive,Electricity,NaN,NaN
9213,2002,Ford,Explorer USPS Electric,NaN,2-Wheel Drive,Electricity,NaN,NaN
10329,2003,Toyota,RAV4 EV,NaN,2-Wheel Drive,Electricity,NaN,NaN
21413,1985,Subaru,RX Turbo,Manual 5-spd,4-Wheel Drive,Regular,NaN,NaN


In [13]:
#Fill missing vales with nulls:
data[['displ', 'cylinders']] = data[['displ', 'cylinders']].fillna(0)

In [17]:
#Examine other columns with missing values:
null_trany = data[(data['trany'].isnull()==True)]
null_trany[['year', 'make', 'model', 'trany', 'drive','fuelType','cylinders', 'displ']]

,year,make,model,trany,drive,fuelType,cylinders,displ
7138,2000,Nissan,Altra EV,NaN,NaN,Electricity,0.0,0.0
7139,2000,Toyota,RAV4 EV,NaN,2-Wheel Drive,Electricity,0.0,0.0
8143,2001,Toyota,RAV4 EV,NaN,2-Wheel Drive,Electricity,0.0,0.0
8144,2001,Ford,Th!nk,NaN,NaN,Electricity,0.0,0.0
8146,2001,Ford,Explorer USPS Electric,NaN,2-Wheel Drive,Electricity,0.0,0.0
8147,2001,Nissan,Hyper-Mini,NaN,NaN,Electricity,0.0,0.0
9212,2002,Toyota,RAV4 EV,NaN,2-Wheel Drive,Electricity,0.0,0.0
9213,2002,Ford,Explorer USPS Electric,NaN,2-Wheel Drive,Electricity,0.0,0.0
10329,2003,Toyota,RAV4 EV,NaN,2-Wheel Drive,Electricity,0.0,0.0
19097,1984,Ford,F150 Pickup 2WD,NaN,2-Wheel Drive,Regular,8.0,5.8


In [18]:
#Examine other columns with missing values:
null_drive = data[(data['drive'].isnull()==True)]
null_drive[['year', 'make', 'model', 'trany', 'drive','fuelType','cylinders', 'displ']]

,year,make,model,trany,drive,fuelType,cylinders,displ
7138,2000,Nissan,Altra EV,NaN,NaN,Electricity,0.0,0.0
8144,2001,Ford,Th!nk,NaN,NaN,Electricity,0.0,0.0
8147,2001,Nissan,Hyper-Mini,NaN,NaN,Electricity,0.0,0.0
18217,1984,Alfa Romeo,Spider Veloce 2000,Manual 5-spd,NaN,Regular,4.0,2.0
18218,1984,Bertone,X1/9,Manual 5-spd,NaN,Regular,4.0,1.5
18219,1984,Chevrolet,Corvette,Automatic 4-spd,NaN,Regular,8.0,5.7
18220,1984,Chevrolet,Corvette,Manual 4-spd,NaN,Regular,8.0,5.7
18221,1984,Nissan,300ZX,Automatic 4-spd,NaN,Regular,6.0,3.0
18222,1984,Nissan,300ZX,Automatic 4-spd,NaN,Regular,6.0,3.0
18223,1984,Nissan,300ZX,Manual 5-spd,NaN,Regular,6.0,3.0


In [19]:
#Incorrect Values; identifying rows where the correlation between certain columns is not met
#Try to find the missing value by analysing the incorrect value field and it's reference fields

test = data[(data['cylinders']==0) & (data['displ']!=0)]
test[['year', 'make', 'model', 'trany', 'drive','fuelType','cylinders', 'displ']]

,year,make,model,trany,drive,fuelType,cylinders,displ
21506,1986,Mazda,RX-7,Manual 5-spd,Rear-Wheel Drive,Regular,0.0,1.3


In [25]:
#Can fill in the missing value field with a value that we've found:
data.loc[(data['cylinders']==0) & (data['displ']!=0), 'cylinders'] = 4

In [27]:
#Low Variance Columns; identifying columns where 90% of the data is the same:

import numpy as np
low_variance = []

for col in data._get_numeric_data():
    minimum = min(data[col])
    ninety_perc = np.percentile(data[col], 90)
    if ninety_perc == minimum:
        low_variance.append(col)

print(low_variance)

['barrelsA08', 'charge120', 'charge240', 'cityA08', 'cityA08U', 'cityCD', 'cityE', 'cityUF', 'co2A', 'co2TailpipeAGpm', 'combA08', 'combA08U', 'combE', 'combinedCD', 'combinedUF', 'fuelCostA08', 'ghgScoreA', 'highwayA08', 'highwayA08U', 'highwayCD', 'highwayE', 'highwayUF', 'phevBlended', 'range', 'rangeCity', 'rangeCityA', 'rangeHwy', 'rangeHwyA', 'UCityA', 'UHighwayA', 'charge240b', 'phevCity', 'phevHwy', 'phevComb']


In [28]:
#Removing these columns:
data = data.drop(low_variance, axis=1)

In [30]:
#Extreme Values and Outliers:

#describe() function gives the description of data in each column(mean, min, max, IQR etc), 
#and transpose() allows to have all the initial data column names on left as rows and description items as columns
#we use the IQR column to identify percentiles

stats = data.describe().transpose()
stats['IQR'] = stats['75%'] - stats['25%']
stats


,count,mean,std,min,25%,50%,75%,max,IQR
barrels08,37843.0,17.532506,4.575950,0.06,14.33087,17.347895,20.600625,47.087143,6.269755
city08,37843.0,17.941389,6.660360,6.00,15.00000,17.000000,20.000000,138.000000,5.000000
city08U,37843.0,4.042737,9.645820,0.00,0.00000,0.000000,0.000000,138.304000,0.000000
co2,37843.0,61.503713,153.387715,-1.00,-1.00000,-1.000000,-1.000000,847.000000,0.000000
co2TailpipeGpm,37843.0,473.179736,122.188847,0.00,388.00000,467.736842,555.437500,1269.571429,167.437500
comb08,37843.0,20.195809,6.623444,7.00,17.00000,19.000000,23.000000,124.000000,6.000000
comb08U,37843.0,4.549751,10.389994,0.00,0.00000,0.000000,0.000000,124.360100,0.000000
cylinders,37843.0,5.719446,1.778959,0.00,4.00000,6.000000,6.000000,16.000000,2.000000
displ,37843.0,3.307658,1.371982,0.00,2.20000,3.000000,4.300000,8.400000,2.100000
engId,37843.0,8860.308961,17829.683477,0.00,0.00000,211.000000,4505.000000,69102.000000,4505.000000


In [34]:
#Create an empty data frame called outliers with the same columns as initial data set. 
#Loop through each column in the data calculating the lower and upper bounds, 
#retrieving records where the value for that column falls outside the bounds we established, 
#and appending those results to our outlier data frame:

outliers = pd.DataFrame(columns=data.columns)

for col in stats.index:
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 1.5
    lower = stats.at[col,'25%'] - cutoff
    upper = stats.at[col,'75%'] + cutoff
    results = data[(data[col] < lower) | 
                   (data[col] > upper)].copy()
    results['Outlier'] = col #this added column will show the initial data set column name that contains the outlier
    outliers = outliers.append(results)

In [35]:
#Outliers data frame is now populated with records that can be investigated further 
#to determine whether they should be kept in the data or dropped. 
outliers.head()

,Outlier,UCity,UHighway,VClass,barrels08,city08,city08U,co2,co2TailpipeGpm,comb08,...,lv4,make,model,modifiedOn,mpgData,pv2,pv4,trany,year,youSaveSpend
47,barrels08,11.0,15.0,Vans,32.961,9,0.0,-1,888.7,10,...,0,Dodge,B150/B250 Wagon 2WD,Tue Jan 01 00:00:00 EST 2013,N,0,0,Automatic 3-spd,1985,-10000
58,barrels08,12.0,15.0,Vans,32.961,10,0.0,-1,888.7,10,...,0,Dodge,B350 Wagon 2WD,Tue Jan 01 00:00:00 EST 2013,N,0,0,Automatic 3-spd,1985,-10000
69,barrels08,12.0,15.0,Vans,32.961,10,0.0,-1,888.7,10,...,0,Dodge,B350 Wagon 2WD,Tue Jan 01 00:00:00 EST 2013,N,0,0,Automatic 3-spd,1985,-10000
80,barrels08,11.0,14.0,Vans,32.961,9,0.0,-1,888.7,10,...,0,Dodge,B350 Wagon 2WD,Tue Jan 01 00:00:00 EST 2013,N,0,0,Automatic 3-spd,1985,-10000
275,barrels08,11.0,16.0,Standard Pickup Trucks,32.961,9,0.0,-1,888.7,10,...,0,Chevrolet,C1500 Pickup 2WD,Tue Jan 01 00:00:00 EST 2013,N,0,0,Automatic 4-spd,1993,-10000


In [36]:
#Data Type Correction
#Check data type for each column:
data.dtypes

barrels08         float64
city08              int64
city08U           float64
co2                 int64
co2TailpipeGpm    float64
comb08              int64
comb08U           float64
cylinders         float64
displ             float64
drive              object
engId               int64
feScore             int64
fuelCost08          int64
fuelType           object
fuelType1          object
ghgScore            int64
highway08           int64
highway08U        float64
hlv                 int64
hpv                 int64
id                  int64
lv2                 int64
lv4                 int64
make               object
model              object
mpgData            object
pv2                 int64
pv4                 int64
trany              object
UCity             float64
UHighway          float64
VClass             object
year                int64
youSaveSpend        int64
createdOn          object
modifiedOn         object
dtype: object

In [40]:
#Change data type of a column:
data['year'] = data['year'].astype('object') 
data['year'].dtype


dtype('O')

In [41]:
#Cleaning Text and Removing Special Characters
#Check unique values of a column:
print(set(data['trany']))

#some values are not grouped due to special characters

{nan, 'Manual 5-spd', 'Automatic 9-spd', 'Auto(AM6)', 'Auto(AV-S8)', 'Manual 5 spd', 'Automatic (A6)', 'Auto(A1)', 'Automatic (S7)', 'Manual 3-spd', 'Automatic 7-spd', 'Manual 4-spd Doubled', 'Auto(AV-S7)', 'Automatic (A1)', 'Auto (AV)', 'Automatic 6spd', 'Auto(L3)', 'Manual(M7)', 'Manual 7-spd', 'Automatic (AM5)', 'Automatic (S8)', 'Automatic (AV-S6)', 'Auto(AM-S8)', 'Auto(AM5)', 'Auto (AV-S8)', 'Automatic 6-spd', 'Auto(AV-S6)', 'Automatic (S5)', 'Auto (AV-S6)', 'Auto(AM8)', 'Automatic 5-spd', 'Automatic (S6)', 'Auto(AM-S9)', 'Auto(L4)', 'Automatic 4-spd', 'Automatic (S4)', 'Automatic (S9)', 'Automatic (AM6)', 'Auto(AM-S6)', 'Manual 4-spd', 'Automatic (AV)', 'Automatic 3-spd', 'Auto(AM7)', 'Automatic 8-spd', 'Automatic (variable gear ratios)', 'Manual 6-spd', 'Auto(AM-S7)'}


In [42]:
#Replace special characters:
data['trany'] = data['trany'].str.replace('-', '') 
print(set(data['trany']))

{nan, 'Automatic 5spd', 'Manual 4spd', 'Automatic 4spd', 'Auto(AM6)', 'Auto(A1)', 'Manual 5 spd', 'Automatic (A6)', 'Automatic (S7)', 'Automatic (A1)', 'Auto (AV)', 'Auto(AVS7)', 'Auto(AMS7)', 'Auto(L3)', 'Manual(M7)', 'Manual 5spd', 'Automatic 8spd', 'Automatic (AVS6)', 'Automatic 7spd', 'Auto(AVS8)', 'Auto(AMS6)', 'Auto(AVS6)', 'Automatic (AM5)', 'Auto(AMS8)', 'Automatic (S8)', 'Auto(AM5)', 'Automatic 9spd', 'Automatic (S5)', 'Manual 7spd', 'Manual 3spd', 'Auto(AM8)', 'Automatic 3spd', 'Automatic (S6)', 'Manual 4spd Doubled', 'Auto(AMS9)', 'Manual 6spd', 'Auto(L4)', 'Automatic (S4)', 'Automatic (S9)', 'Automatic (AM6)', 'Automatic (AV)', 'Automatic (variable gear ratios)', 'Auto (AVS6)', 'Auto(AM7)', 'Automatic 6spd', 'Auto (AVS8)'}


In [43]:
#Replace other charachters, or abrievations to make data more homogeneous
data['trany'] = data['trany'].str.replace('Automatic', 'Auto')
data['trany'] = data['trany'].str.replace('Auto\(', 'Auto ')
data['trany'] = data['trany'].str.replace('Manual\(', 'Manual ')
data['trany'] = data['trany'].str.replace('\(', '')
data['trany'] = data['trany'].str.replace('\)', '')
print(set(data['trany']))

#This allows to reduce the number of unique values in the column

{nan, 'Manual M7', 'Auto variable gear ratios', 'Manual 4spd', 'Manual 5 spd', 'Auto 8spd', 'Auto 5spd', 'Manual 5spd', 'Auto AMS6', 'Auto AMS7', 'Auto S5', 'Auto S8', 'Auto S7', 'Auto L4', 'Auto S6', 'Auto AV', 'Auto S9', 'Auto 3spd', 'Auto AM6', 'Auto AVS6', 'Auto A6', 'Auto AVS8', 'Auto AM8', 'Manual 7spd', 'Auto 7spd', 'Auto 4spd', 'Auto AMS9', 'Auto 6spd', 'Manual 3spd', 'Auto L3', 'Auto AVS7', 'Manual 4spd Doubled', 'Auto S4', 'Manual 6spd', 'Auto AM5', 'Auto 9spd', 'Auto AMS8', 'Auto A1', 'Auto AM7'}


In [44]:
#Finding and Removing Duplicates
#First use drop_duplicates method to remove identical rows:
before = len(data)
data = data.drop_duplicates()
after = len(data)
print('Number of duplicate records dropped: ', str(before - after))


Number of duplicate records dropped:  0


In [45]:
#We can chose to drop duplicates based on certain columns only:
select_columns = ['make', 'model', 'year', 'displ', 'cylinders', 'trany', 'drive', 
                  'VClass','fuelType','barrels08', 'city08', 'highway08', 'comb08', 
                  'co2TailpipeGpm', 'fuelCost08'] 
data = data[select_columns].drop_duplicates() 
after = len(data) 
print('Number of duplicate records dropped: ', str(before - after))

Number of duplicate records dropped:  885
